# python 实现PCFG

本例子主要受 Michael Collins 教授的 Probabilistic Context-Free Grammars (PCFGs) 启发而编写，为了帮助大家理解，我在我的博客、公众号上发表了文章[一文读懂NLP中的PCFG(公众号)](https://mp.weixin.qq.com/s?__biz=MzIwNDM1NjUzMA==&mid=2247483666&idx=1&sn=708dcbce5be808b3be273838db298da7&chksm=96c02fcfa1b7a6d99a69c35e0de413488d4da4dc13c4ab3d21c8a415c8f2310c141676a068e0#rd)，欢迎大家阅读。当然强烈推荐Michael Collins 教授的 [Probabilistic Context-Free Grammars (PCFGs)](http://www.cs.columbia.edu/~mcollins/courses/nlp2011/notes/pcfgs.pdf)

pcfg 常用于生成文法解析树，再这里使用CKY算法对CNF（Chomsky Normal Form）的文法进行解析

## 目录

1. [项目结构](#项目结构)
2. [环境要求](#环境要求)
3. [代码分析](#代码分析)
4. [项目后续](#项目后续)
5. [联系作者](#联系作者)

## 项目结构

| - src

    | - corpus        语料库

    | - pcfg.py       

    | - main.py       例子程序

## 环境要求

    python3

## 代码分析

### pcfg.py


In [1]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

# -------------------------------------------#
# PCFG Parser	                             #
# author: sean lee                           #
# qq: 929325776							     #
# email: lxm_0828@163.com                    #
#--------------------------------------------#

from collections import defaultdict

class PCFG(object):

	# N_dict - count nonterminal
	# NR_dict - count relation X->Y1 Y2 (X Y1 Y2 are nonterminal)
	# TR_dict - count relation X->y (X is nonterminal y is terminal)
	def __init__(self):
		self.N_dict = defaultdict(int)
		self.NR_dict = defaultdict(int)
		self.TR_dict = defaultdict(int)

	def fit(self, train_corpus):
		with open(train_corpus, 'r') as f:
			for line in f:
				arr = line.strip().split('->')
				self.N_dict[arr[0]] += 1;
				if ' ' in arr[1].strip():
					arr2 = arr[1].split()
					if len(arr2) > 2:
						continue
					self.N_dict[arr2[0]] += 1
					self.N_dict[arr2[1]] += 1
					self.NR_dict[(arr[0], arr2[0], arr2[1])] += 1
				else:
					self.TR_dict[(arr[0], arr[1])] += 1
	# q(X->Y Z)
	def calc_NR_proba(self, x, y1, y2):
		return float(self.NR_dict[(x, y1, y2)]) / self.N_dict[x]

	# q(X->y)
	def calc_TR_proba(self, x, y):
		return float(self.TR_dict[(x, y)]) / self.N_dict[x]

	# Return parse tree
	def parse(self, sentence):
		import json
		print(json.dumps(self.CKY(sentence.split())))

	# CKY algorithm 
	# 适用于CNF (Chomsky normal form)
	def CKY(self, sentence):
		n = len(sentence)
		pi = defaultdict(float) 
		bp = {}	# backpointer
		N = self.N_dict.keys()

		for i in range(n):
			word = sentence[i]
			for X in N:
				pi[(i, i, X)] = self.calc_TR_proba(X, word)

		for i in range(1, n):
			for j in range(n-1):
				k = i + j
				for X in N:
					max_score = 0
					argmax = None
					for R in self.NR_dict.keys():
						if R[0] == X:  # start from X
							Y, Z = R[1:]
							for s in range(j, k):
								if pi[(j, s, Y)] and pi[s+1, k, Z]:
									score = self.calc_NR_proba(X, Y, Z) * pi[(j, s, Y)] * pi[s+1, k, Z]
									if max_score < score:
										max_score = score
										argmax = Y, Z, s
					if max_score:
						pi[j, k, X] = max_score
						bp[j, k, X] = argmax

		# return
		if pi[(0, n-1, 'S')]:
			return self.recover(sentence, bp, 0, n-1, 'S')
		else:
			max_score = 0
			argmax = 0, 0, 'S'
			for X in N:
				if max_score < pi[(0, n-1, X)]:
					max_score = pi[(0, n-1, X)]
					argmax = 0, n-1, X
			return self.recover(sentence, bp, *argmax)

	#  Return the list of the parsed tree with back pointers.
	def recover(self, sentence, bp, i, j, X):
		if i == j:
			return [X, sentence[i]]
		else:
			Y, Z, s = bp[i, j, X]
			return [X, self.recover(sentence, bp, i, s, Y), self.recover(sentence, bp, s+1, j, Z)]

### main.py

In [7]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-

# -------------------------------------------#
# main.py    	                             #
# author: sean lee                           #
# qq: 929325776							     #
# email: lxm_0828@163.com                    #
#--------------------------------------------#

parser = PCFG()
parser.fit('./corpus/toy/train.txt')

'''
print(parser.N_dict)
print(parser.NR_dict)
print(parser.TR_dict)
'''

sentence = "the man saw the dog"
print("sentence:", sentence)
print("parse tree")
parser.parse(sentence)

sentence: the man saw the dog
parse tree
["S", ["NP", ["DT", "the"], ["NN", "man"]], ["VP", ["Vt", "saw"], ["NP", ["DT", "the"], ["NN", "dog"]]]]


## 项目后续

过段时间会加入深度学习在NLP上的应用，如果你感兴趣，可以关注我的公众号，或者star, watch 本项目哦

## 联系作者

@author sean

@qq 929325776

有什么问题，可以联系我，一起讨论